In [1]:
# def checkResp(Res,rq_ResFn,rq_ResSFn):
#     Res = Res.decode()
#     Res_Fn = Res[0]+Res[1]
#     Res_SFn = Res[3]+Res[4]
# a= 8
# len(str(a))
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

def encrypt_AES(Cryptkey, data):
    print(data)
    print(len(data))
    data = bytes.fromhex(data)
    cipher = AES.new(Cryptkey, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv
    return iv + ciphertext

def decrypt_AES(key, encrypted_data):
    iv = encrypted_data[:AES.block_size]
    ciphertext = encrypted_data[AES.block_size:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

# Example usage
# key = get_random_bytes(16)  # 16 bytes key for AES-128
Cryptkey = b'\xb9t5RR\x9d\x1d+\xaa\xa3\xd1}\xfb\x1a\x1d\xcb'
# data = b"Hello, world! This is AES encryption example."

# encrypted_data = encrypt_AES(key, data)
# print("Encrypted data:", encrypted_data)

# decrypted_data = decrypt_AES(key, encrypted_data)
# print("Decrypted data:", decrypted_data.decode())

In [2]:
# print(decrypt_AES(Cryptkey,encrypt_AES(Cryptkey,Cryptkey)))

In [3]:
def KeyGen(seed):
    global Cryptkey
    return encrypt_AES(Cryptkey, seed).hex()
def ValLevel(lvl):
    if len(lvl)==1:
        return '0'+lvl
    else:
        return lvl
def ExtractKS (from_server, secAccLvl):
#     print(from_server[0:2])

    if from_server[0:2]  == '67' and int(from_server[3:5])%2 == 0: 
        print(from_server)
        print('ECU unlocked sucessfully')
        return 1
    if from_server[0:2]  == '67' and int(from_server[3:5])%2 == 1:
        return '27 ' + ValLevel(str(secAccLvl+1)) + ' '+KeyGen(from_server[6:len(from_server)])
    return -1
# print(ExtractKS('67 01 67 68 69'))

In [4]:
import socket
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('127.0.0.1', 8080))
closeCon = 0
client.send("Tester Connected\n".encode())
secAccLvl = '07'
while closeCon ==0 :
    from_server = client.recv(4096)
    if from_server.decode() == 'End':
        closeCon = 1
        break
    print (from_server.decode())
    if int(secAccLvl)%2 == 1:
        resp = '27 '+ secAccLvl
        print(resp)
        client.send(resp.encode())
        from_server = client.recv(4096).decode()
        print (from_server)
    resp = ExtractKS(from_server,int(secAccLvl))
    if resp == -1:
        print('error in message Fn or SubFn')
        closeCon = 1
        client.send(resp.encode())
        break
    elif resp == 1:
        print("ECU Unlocked")
        closeCon = 1
        client.send(resp.encode())
        break
    else:
        secAccLvl = str(int(secAccLvl)+1)
        closeCon = 1
        print(resp)
        client.send(resp.encode())
        from_server = client.recv(4096).decode()
        resp = ExtractKS(from_server,int(secAccLvl))
client.close()



ECU Connected
27 07
67 07 387f3460
387f3460
8
27 08 0f051d30869256481ba12b425ddbe87d9fa1340df57d8c8b1ecf895bb8708632
67 08
ECU unlocked sucessfully


In [ ]:
from_server = '213456789'
print(from_server[5:9])

In [ ]:
# ExtractKS('67 01 51 52 54')
# def ExtractKS (from_server):
#     if from_server[0:1]  == '67' and int(from_server[3:4])%2 == 0: 
#         return from_server[5:len(from_server)]
#     if from_server[0:1]  == '27' and int(from_server[3:4])%2 == 1:
#         return from_server[5:len(from_server)]
#     return 0